## INTEGER CODING RULES
 
**1.** Replace the ```"return"``` statement in each function with one or more lines of C code that implements the function. Your code must conform to the following style:
```c
  int Funct(arg1, arg2, ...) {
      /* brief description of how your implementation works */
      int var1 = Expr1;
      ...
      int varM = ExprM;

      varJ = ExprJ;
      ...
      varN = ExprN;
      return ExprR;
  }
```


Each ```"Expr"``` is an expression using ONLY the following:

* 1. Integer constants ```0 through 255 (0xFF)```, inclusive. You are
  not allowed to use big constants such as ```0xffffffff```.
* 2. Function arguments and ```local``` variables ```(no global variables)```.
* 3. Unary integer operations ```! ~```
* 4. Binary integer operations ```& ^ | + << >>```

## A. Difficulty level 1
### 1. BitAnd using only ~ and |
**Recall Demorgan's law:**
<img src="resources/demorgan.png" alt="Drawing" style="width: 180px;"/>
Demorgan's law indicates: 
```a & b = ~( ~a | ~b)```

In [22]:
int bitAnd(int x, int y) {
  /* x & y is equivalent to the negation of (~x | ~y) (DeMorgan's Law) */
  return ~(~x | ~y);
}

### 2. BitXor using only ~ and &
XOR: ```a ^ b = a & ~b + ~a & b```

Using Demorgan's law
```a ^ b = ~(~(a & ~b) & ~(~a & b))```

In [23]:
int bitXor(int x, int y) {
  /*
   * x^y is equivalent to (x & ~y) | (~x & y),
   * which is also equivalent to ~(~(x & ~y) & ~(~x & y))
   */
  return ~(~(x & ~y) & ~(~x & y));
}

### 3.  ThirdBits - return word with every third bit (starting from the LSB) set to 1
*   Legal ops: ! ~ & ^ | + << >>
*   Max ops: 8

In [24]:
long thirdBits(void) {
  /*
   * Sets x equal to 1 then shifts it over 3 and adds 1, 
   * then shifts that over 6 and adds the 3 shifted one,
   * then shifts that over 12 and adds the 6 shifted one,
   * then shifts that over 24 and adds the 12 shifted one
   */
  long x = 1;
  x = (x << 3) + 1;
  x = x + (x << 6);
  x = x + (x << 12);
  x = x + (x << 24);
  x = x + (x << 48);
  return x;
}

In [25]:
int res = 0;
res = thirdBits();
printf("res=%d", res);

res=1227133513

## B. Difficulty level 2
### 1. Fits bits for 2's complement
Return 1 if x can be represented as an n-bit, two's complement integer, where 1<=n<=32

**Note that in C, left shift fills 0, right shift fills the signed bit for signed numbers**

Therefore, shift x left by ```(32-n)``` bits, if it cannot be accomodated, the MSB changes **from 0 to 1 or from 1 to 0**
* When x is positive and cannot be accomodated: **MSB changes from 0 to 1**
* When x is negative and cannot be accomodated: **MSB changes from 1 to 0**

When shift x back to right, value of x will change if x cannot be represented as an n-bit

In [26]:
int fitsBits(int x, int n) {
  /*
   * Shift x to the left by (32 - n) bits and then shift back.
   * If the value of x remains unchanged, then x can be represented
   * as an n-bit.
   */
  int shift = 32 + (~n + 1);
  int num = x;
  num = num << shift;
  num = num >> shift;
  return !(num ^ x);
}

In [27]:
int res = 0;
res = fitsBits(-5, 3);
printf("res=%d", res);

res=0

#### 1. a. Quick Note: Equal with xor
Bitwise XOR using operands ```a``` and ```b``` produce 0 if ```a == b``` 
Therefore, expressions below are equivalent:
```c
res1 = !(a^b);
res2 = (a == b);
```

In [28]:
int a = 10;
int b = 10;
int c = 9;
int res1, res2;
res1 = !(a ^ b);
res2 = (a == b);

In [29]:
printf("res1(!(a^b))=%d\n", res1);
printf("res2(a==b)=%d\n", res2);

res1(!(a^b))=1
res2(a==b)=1


### 2. Sign bit for an int number
Return 1 if positive, 0 if zero, and -1 if negative

Again: **Note that in C, right shift fills the signed bit for signed numbers**

Therefore, right shift for 31 bits will keep the **sign bit only**

Meanwhile, we need to seperate 0 from positive as the sign bit is 0 for both positive numbers and zeros

#### 2.a. Quicknote: Determine whether a number is 0 with logic NOT
Consider ```res = !a;```
The logic NOT poperator:
* returns 1 if ```a == 0```
* returns 0 if ```a != 0```
For example

In [30]:
int a = 0;
printf("a is zero, returns 1, !a=%d\n", !a);
int b = 37211;
printf("b is non-zero, returns 0, !b=%d\n", !b);

a is zero, returns 1, !a=1
b is non-zero, returns 0, !b=0


On the contary, ```~``` bitwise NOT negatates every bit of the operand, returns -a - 1 (2's complement - 1)

In [31]:
int a = 37212;
printf("~a=%d", ~a);

~a=-37213

Therefore, the solution will be: ```shift_right_31bit(a) | !(!a)```

In [32]:
int sign(int x) {
  return (x >> 31) | (!(!a));
 }

Why shifting more bits doesn't work? **Overflow** is causing problems right here

In [33]:
int a = -2147483647;

In [34]:
printf("rsh32=%d\n", a>>32);

input_line_40:2:24: warning: shift count >= width of type [-Wshift-count-overflow]
 printf("rsh32=%d\n", a>>32);
                       ^ ~~


rsh32=-2147483647


In this case, Shifting by 31 bit is correct and works. 

In [35]:
printf("rsh31=%d\n", a>>31);

rsh31=-1


### 3. Get byte n from word x
Assume it is a little endian machine, Byte numbered from 0(LSB) to 3(MSB). 

#### 3.a.Quicknote: Filter values with Bitwise AND

* Bitwise AND an operand with 0xFF will retain 1's in ```LSB[7:0]```
* Bitwise AND an operand with 0x0 will clear that number to all zeros

Shift right the input number by ```n*8``` bit, move the needed byte to ```LSB[7:0]```.

Then bitwise AND the number with 0xFF to get the lower eight bits.

In [36]:
int getByte(int x, int n) {
    /*
     * Shift the word to right by 8*n bits and then
     * use & with 0xFF to keep the byte we want. 
     */
    int shift = n << 3;
    return (x >> shift) & 0xFF;
}

## C. Difficulty level 3
### 1. LogicalShift

*   Can assume that 0 <= n <= 31
*   Examples: logicalShift(0x87654321,4) = 0x08765432

Logic shift right fills 0 into MSB, then we need to create a mask to filter 1's. The mask will be like bitwise AND with 0xF
* When n == 0: Mask is 0xFFFFFFFF, i.e. retain x's original value
* Otherwise, Mask is 0x7FFFFFFF **shifted right** by **```n-1 bit```**

In [37]:
int logicalShift(int x, int n) {
  /*
   * Creates a filter to get rid of any leading 1's created by
   * the arithmetic shift to make it become logical shift. The
   * value of filter is 0 followed by all 1's, and then it's
   * shifted to the right by n - 1 bits. Shift x to the right by
   * n bits and use & with filter to get the result of logical shift.
   */
  int filter;
  filter = 0x7f;
  filter = (filter << 8) + 0xFF;
  filter = (filter << 8) + 0xFF;
  filter = (filter << 8) + 0xFF;
//   filter = ((filter >> n) << 1) + 1;
  filter = (n == 0) ? 0xFFFFFFFF: filter >> (n - 1);
  return (x >> n) & filter; 
}

In the above code, Line 15 can be rewritten as
```c
filter = ((filter >> n) << 1) + 1;
```
If there was any constraints

* When n is non-zero, mask is **shift right by n bit, and shift left by 1 bit**, then **add 1** to retain its original LSB value
* When n is zero, mask is left by 1 bit, and add 1, changing mask from ```0x7FFFFFFF to 0xFFFFFFFF```

## 2. AddOk?
Determine whether overflow occurs when perform addition
Function returns 1 if ```x + y``` won't cause overflow

When adding two operands, OV happens when the signs for two numbers are the same, but their sum has a differenct sign

Therefore, the solution will be:
* Get MSBs of op1, op2, and sum
    * Get MSB by **shifting right** 31 bits for ```int```
    * Overflow condition can be expressed as: ```MSB(a)==MSB(b) && MSB(a)!=MSB(sum)```
* Comparing MSBs of op1, op2, sum using XOR operation
    * Recall that ```a^b == 0``` means a is equal to b
* The expression for overflow should be:
    * ```~(MSB(a) ^ MSB(b)) & (MSB(a) ^ MSB(b))```
* In this case, we need to negate the above expression because returning 1 when **NO** Overflow

In [38]:
int addOK(int x, int y) {
  /*
   * Overflow occurs when the sign of x and y are the same, but
   * the sign of x + y is different. Use ! in front of 
   * (~(sign_x ^ sign_y) & (sign_x ^ sign_sum)) so it gives 1
   * when x + y won't ovreflow.
   */
  int sum = x + y;
  int sign_x = x >> 31;
  int sign_y = y >> 31;
  int sign_sum = sum >> 31;
  return !(~(sign_x ^ sign_y) & (sign_x ^ sign_sum));
}

### 3. Conditional Operator
The same as ternary operation: ```x?y:z```

First determine whether x is 0 by using ```!```, declare an intermediate variable a = !!(x)
* When x is non-zero, ```!!x``` returns 32'b1, otherwise returns 32'b0
    * When x is non-zero, ```~a + 1``` is all ones. Result should be y & 0xFFFFFFFF
    * When x is zero, ```~a + 1``` is all zeros, Result should be z & 0xFFFFFFFF

In [21]:
int conditional(int x, int y, int z) {
  /*
   * First determine whether x is 0 using !!x, then use a variable b to hold
   * ~(!!x) + 1, which is all bits 0 when x is 0 and all bits 1 otherwise.
   * Then use | to (b & y) and (~b & y). Since b is either all bits 0 or 1,
   * one of the expression will give 0 and the other one give the original
   * value. | with 0 doesn't change the value.
   */
  int a = !!x;
  int b = ~a + 1;
  return (b & y) | (~b & z);
}

## D. Difficulty level 4
### 1. Compute ```Logical Not``` without using ```!```
Return 1 when operand is 0, otherwise returns 0

Remember that 0 is the only number that ```x - 1 < 0``` and ```~x < 0```

Then get MSB of x - 1 and ~x, truncate the MSB expression by AND with 0x1

In [20]:
int bang(int x) {
  /*
   * Since 0 is the only number where x-1 < 0 and ~x is less than 0,
   * & x-1 and ~x the sign bit will tell whether x is zero or nonzero.
   * Shifting the sign bit down to the first bit (LSB) and & with 1
   * will give 1 if x is zero and zero otherwise.
   */
  int x_minus_one, x_minus_one_msb, x_bar_msb;
  int sign_holder;
  x_minus_one = x + (~1 + 1);
  x_minus_one_msb = x_minus_one >> 31;
  x_bar_msb = (~x) >> 31;
  sign_holder = x_bar_msb & x_minus_one_msb;
  return sign_holder & 0x1;
 }

### 2. IsPower2
Returns 1 if x is power of 2, note in this problem, 0 is not considered as pow2
Note that:
* ```!(1) = 0```
* ```!(0) = 1```

If x is a pow2, then ```(x - 1) & x``` is 0
* Note that ```x - 1``` can be expressed as ```x + ~1 + 1```
* ```!(x - 1)&x = 1``` if there was no 1s

Meanwhile, we should check whether x is 0 or negative
* ```!(x>>31) = 1``` when x is non negative 
* ```!(x) = 1``` when x is 0
* Then we can rewrite the above as:
    * ```!(x>>31) = 1``` if x is non-negative, ```=0``` if x is negative
    * ```!!(x) = 1``` when x is non-zero, ```=0``` if x is zero 
    
X is pow2 when:
1. No 1s for (x - 1) & x
2. Non-negative
3. Non-zero

In [ ]:
int isPower2(int x) {
  /*
   * Variable a checks if x is power of 2, x and x - 1 won't have a 1
   * in the same place if it's power of 2. Variable b checks if x is
   * negative or zero. Use & to combine !a and b to complete the function.
   */
  int non1s = !(x & (x + ~1 + 1));
  int non_negative = !(x >> 31);
  int non_zero = !(!(x));
  return non1s & non_negative & non_zero;
}

### 3. Invert certain bits
Q: Given an integer ```x```, bit flip ```n``` bits starting at position ```p```

1. Using ```x^1``` can flip a bit:
* For example:
    * ```x = 0: x ^ 1 = 1```
    * ```x = 1: x ^ 1 = 0```
    
2. Then we need a **mask** to control which bits to flip
* The mask should contain n 1's at lower n bits, then **left shift n bits by p**
    * Create a all one number ~(0x0), i.e. ```-1```, shift that ```all ones number``` left by n bits, and flip that number using **NOT** or **XOR with ONES**. Get a number with ```n x 1s``` at lower n bits
    * Shift that n x 1s number left by p bits, XOR with the target operand

In [3]:
int invert(int x, int p, int n) {
  int mask = (-1)^(-1 << n); // create 0b000....111111.111;
  mask = mask << p; // Shift left to get 0b000....111...000;
  int res;
  res = x ^ mask; // res = mask ^ 
  return res;
}

In [4]:
printf("invert (%d, %d, %d) = %d", 1645, 2, 4, invert(1645, 2, 4));

invert (1645, 2, 4) = 1617